
<br>
===========<br>
Skeletonize<br>
===========<br>
Skeletonization reduces binary objects to 1 pixel wide representations. This<br>
can be useful for feature extraction, and/or representing an object's topology.<br>
``skeletonize`` works by making successive passes of the image. On each pass,<br>
border pixels are identified and removed on the condition that they do not<br>
break the connectivity of the corresponding object.<br>


In [ ]:
from skimage.morphology import skeletonize
from skimage import data
import matplotlib.pyplot as plt
from skimage.util import invert

Invert the horse image

In [ ]:
image = invert(data.horse())

perform skeletonization

In [ ]:
skeleton = skeletonize(image)

display results

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8, 4),
                         sharex=True, sharey=True)

In [ ]:
ax = axes.ravel()

In [ ]:
ax[0].imshow(image, cmap=plt.cm.gray)
ax[0].axis('off')
ax[0].set_title('original', fontsize=20)

In [ ]:
ax[1].imshow(skeleton, cmap=plt.cm.gray)
ax[1].axis('off')
ax[1].set_title('skeleton', fontsize=20)

In [ ]:
fig.tight_layout()
plt.show()

####################################################################<br>
**Zhang's method vs Lee's method**<br>
<br>
``skeletonize`` [Zha84]_ works by making successive passes of<br>
the image, removing pixels on object borders. This continues until no<br>
more pixels can be removed.  The image is correlated with a<br>
mask that assigns each pixel a number in the range [0...255]<br>
corresponding to each possible pattern of its 8 neighbouring<br>
pixels. A look up table is then used to assign the pixels a<br>
value of 0, 1, 2 or 3, which are selectively removed during<br>
the iterations.<br>
<br>
``skeletonize(..., method='lee')`` [Lee94]_ uses an octree data structure<br>
to examine a 3x3x3 neighborhood of a pixel. The algorithm proceeds by<br>
iteratively sweeping over the image, and removing pixels at each iteration<br>
until the image stops changing. Each iteration consists of two steps: first,<br>
a list of candidates for removal is assembled; then pixels from this list<br>
are rechecked sequentially, to better preserve connectivity of the image.<br>
<br>
Note that Lee's method [Lee94]_ is designed to be used on 3-D images, and<br>
is selected automatically for those. For illustrative purposes, we apply<br>
this algorithm to a 2-D image.<br>
<br>
.. [Zha84] A fast parallel algorithm for thinning digital patterns,<br>
           T. Y. Zhang and C. Y. Suen, Communications of the ACM,<br>
           March 1984, Volume 27, Number 3.<br>
<br>
.. [Lee94] T.-C. Lee, R.L. Kashyap and C.-N. Chu, Building skeleton models<br>
           via 3-D medial surface/axis thinning algorithms.<br>
           Computer Vision, Graphics, and Image Processing, 56(6):462-478,<br>
           1994.<br>


In [ ]:
import matplotlib.pyplot as plt
from skimage.morphology import skeletonize

In [ ]:
blobs = data.binary_blobs(200, blob_size_fraction=.2,
                          volume_fraction=.35, seed=1)

In [ ]:
skeleton = skeletonize(blobs)
skeleton_lee = skeletonize(blobs, method='lee')

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(8, 4), sharex=True, sharey=True)
ax = axes.ravel()

In [ ]:
ax[0].imshow(blobs, cmap=plt.cm.gray)
ax[0].set_title('original')
ax[0].axis('off')

In [ ]:
ax[1].imshow(skeleton, cmap=plt.cm.gray)
ax[1].set_title('skeletonize')
ax[1].axis('off')

In [ ]:
ax[2].imshow(skeleton_lee, cmap=plt.cm.gray)
ax[2].set_title('skeletonize (Lee 94)')
ax[2].axis('off')

In [ ]:
fig.tight_layout()
plt.show()

####################################################################<br>
**Medial axis skeletonization**<br>
<br>
The medial axis of an object is the set of all points having more than one<br>
closest point on the object's boundary. It is often called the *topological<br>
skeleton*, because it is a 1-pixel wide skeleton of the object, with the same<br>
connectivity as the original object.<br>
<br>
Here, we use the medial axis transform to compute the width of the foreground<br>
objects. As the function ``medial_axis`` returns the distance transform in<br>
addition to the medial axis (with the keyword argument ``return_distance=True``),<br>
it is possible to compute the distance to the background for all points of<br>
the medial axis with this function. This gives an estimate of the local width<br>
of the objects.<br>
<br>
For a skeleton with fewer branches, ``skeletonize`` should be preferred.

In [ ]:
from skimage.morphology import medial_axis, skeletonize

Generate the data

In [ ]:
blobs = data.binary_blobs(200, blob_size_fraction=.2,
                          volume_fraction=.35, seed=1)

Compute the medial axis (skeleton) and the distance transform

In [ ]:
skel, distance = medial_axis(blobs, return_distance=True)

Compare with other skeletonization algorithms

In [ ]:
skeleton = skeletonize(blobs)
skeleton_lee = skeletonize(blobs, method='lee')

Distance to the background for pixels of the skeleton

In [ ]:
dist_on_skel = distance * skel

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(8, 8), sharex=True, sharey=True)
ax = axes.ravel()

In [ ]:
ax[0].imshow(blobs, cmap=plt.cm.gray)
ax[0].set_title('original')
ax[0].axis('off')

In [ ]:
ax[1].imshow(dist_on_skel, cmap='magma')
ax[1].contour(blobs, [0.5], colors='w')
ax[1].set_title('medial_axis')
ax[1].axis('off')

In [ ]:
ax[2].imshow(skeleton, cmap=plt.cm.gray)
ax[2].set_title('skeletonize')
ax[2].axis('off')

In [ ]:
ax[3].imshow(skeleton_lee, cmap=plt.cm.gray)
ax[3].set_title("skeletonize (Lee 94)")
ax[3].axis('off')

In [ ]:
fig.tight_layout()
plt.show()

####################################################################<br>
**Morphological thinning**<br>
<br>
Morphological thinning, implemented in the `thin` function, works on the<br>
same principle as `skeletonize`: remove pixels from the borders at each<br>
iteration until none can be removed without altering the connectivity. The<br>
different rules of removal can speed up skeletonization and result in<br>
different final skeletons.<br>
<br>
The `thin` function also takes an optional `max_iter` keyword argument to<br>
limit the number of thinning iterations, and thus produce a relatively<br>
thicker skeleton.

In [ ]:
from skimage.morphology import skeletonize, thin

In [ ]:
skeleton = skeletonize(image)
thinned = thin(image)
thinned_partial = thin(image, max_iter=25)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(8, 8), sharex=True, sharey=True)
ax = axes.ravel()

In [ ]:
ax[0].imshow(image, cmap=plt.cm.gray)
ax[0].set_title('original')
ax[0].axis('off')

In [ ]:
ax[1].imshow(skeleton, cmap=plt.cm.gray)
ax[1].set_title('skeleton')
ax[1].axis('off')

In [ ]:
ax[2].imshow(thinned, cmap=plt.cm.gray)
ax[2].set_title('thinned')
ax[2].axis('off')

In [ ]:
ax[3].imshow(thinned_partial, cmap=plt.cm.gray)
ax[3].set_title('partially thinned')
ax[3].axis('off')

In [ ]:
fig.tight_layout()
plt.show()